In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr


def get_features_num_regression(df, target_col, umbral_corr, pvalue=None):
    """
    Devuelve una lista de variables numéricas cuya correlación con el target
    es mayor en valor absoluto que umbral_corr.

    Argumentos:
    df (pd.DataFrame): DataFrame de entrada.
    target_col (str): Nombre de la variable objetivo numérica.
    umbral_corr (float): Umbral mínimo de correlación (0-1).
    pvalue (float, optional): Nivel de significación estadística.

    Retorna:
    list: Lista de columnas numéricas que cumplen los criterios.
    """

    # --- Checks ---
    if target_col not in df.columns:
        print("Error: target_col no existe en el dataframe.")
        return None

    if not np.issubdtype(df[target_col].dtype, np.number):
        print("Error: target_col debe ser numérica.")
        return None

    if not (0 <= umbral_corr <= 1):
        print("Error: umbral_corr debe estar entre 0 y 1.")
        return None

    if pvalue is not None and not (0 < pvalue < 1):
        print("Error: pvalue debe estar entre 0 y 1.")
        return None

    columnas_validas = []
    num_cols = df.select_dtypes(include=np.number).columns.drop(target_col)

    for col in num_cols:
        corr, p_val = pearsonr(df[col].dropna(), df[target_col].dropna())

        if abs(corr) >= umbral_corr:
            if pvalue is None or p_val <= pvalue:
                columnas_validas.append(col)

    return columnas_validas


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt


def plot_features_num_regression(df, target_col="", columns=None, 
                                 umbral_corr=0, pvalue=None):
    """
    Genera pairplots entre target y variables numéricas
    que cumplan el umbral de correlación y significación.

    Retorna:
    list: columnas que cumplen condiciones.
    """

    if columns is None:
        columns = []

    valid_cols = get_features_num_regression(df, target_col, 
                                             umbral_corr, pvalue)

    if valid_cols is None:
        return None

    if columns:
        valid_cols = [col for col in columns if col in valid_cols]

    if not valid_cols:
        print("No hay variables que cumplan las condiciones.")
        return []

    # EXTRA: máximo 5 variables por gráfico
    chunk_size = 4  # + target = 5 total
    for i in range(0, len(valid_cols), chunk_size):
        cols_chunk = valid_cols[i:i + chunk_size]
        sns.pairplot(df[[target_col] + cols_chunk])
        plt.show()

    return valid_cols


In [ ]:
from scipy.stats import f_oneway, kruskal


def get_features_cat_regression(df, target_col, pvalue=0.05):
    """
    Devuelve variables categóricas cuya relación con target
    sea estadísticamente significativa.

    Argumentos:
    df (pd.DataFrame)
    target_col (str)
    pvalue (float)

    Retorna:
    list
    """

    if target_col not in df.columns:
        print("Error: target_col no existe.")
        return None

    if not np.issubdtype(df[target_col].dtype, np.number):
        print("Error: target_col debe ser numérica.")
        return None

    if not (0 < pvalue < 1):
        print("Error: pvalue debe estar entre 0 y 1.")
        return None

    cat_cols = df.select_dtypes(include=["object", "category"]).columns
    columnas_validas = []

    for col in cat_cols:
        grupos = [df[df[col] == cat][target_col].dropna() 
                  for cat in df[col].dropna().unique()]

        if len(grupos) < 2:
            continue

        try:
            stat, p_val = f_oneway(*grupos)
        except:
            stat, p_val = kruskal(*grupos)

        if p_val <= pvalue:
            columnas_validas.append(col)

    return columnas_validas


In [ ]:
def plot_features_cat_regression(df, target_col="", columns=None,
                                 pvalue=0.05, with_individual_plot=False):
    """
    Grafica histogramas del target agrupados por variables categóricas
    que sean significativas.

    Retorna:
    list
    """

    if columns is None:
        columns = []

    valid_cols = get_features_cat_regression(df, target_col, pvalue)

    if valid_cols is None:
        return None

    if columns:
        valid_cols = [col for col in columns if col in valid_cols]

    if not valid_cols:
        print("No hay variables significativas.")
        return []

    for col in valid_cols:
        plt.figure()
        sns.histplot(data=df, x=target_col, hue=col, multiple="stack")
        plt.title(f"{target_col} vs {col}")
        plt.show()

        if with_individual_plot:
            for cat in df[col].unique():
                plt.figure()
                sns.histplot(df[df[col] == cat][target_col])
                plt.title(f"{target_col} - {col}: {cat}")
                plt.show()

    return valid_cols
